# Abstractive HuggingFace

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/augmentation-abstractive](https://github.com/huseinzol05/Malaya/tree/master/example/augmentation-abstractive).
    
</div>

<div class="alert alert-warning">

Results generated using stochastic methods.
    
</div>

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
%%time

import malaya

CPU times: user 3.02 s, sys: 3.63 s, total: 6.65 s
Wall time: 2.51 s


### Why augmentation

Let say you have a very limited labelled corpus, and you want to add more, but labelling is very costly.

So, text augmentation! We provided few augmentation interfaces in Malaya.

### List available HuggingFace models

In [3]:
malaya.augmentation.abstractive.available_huggingface()

,Size (MB),ROUGE-1,ROUGE-2,ROUGE-L,Suggested length
mesolitica/finetune-noisy-translation-t5-small-bahasa-cased-v4,242.0,0.757218,0.496729,0.304022,256.0
mesolitica/finetune-noisy-translation-t5-base-bahasa-cased-v2,892.0,0.713227,0.470135,0.366797,256.0


### Load HuggingFace model

```python
def huggingface(
    model: str = 'mesolitica/finetune-noisy-translation-t5-small-bahasa-cased-v4',
    lang: str = 'ms',
    force_check: bool = True,
    **kwargs,
):
    """
    Load HuggingFace model to abstractive text augmentation.

    Parameters
    ----------
    model: str, optional (default='mesolitica/finetune-noisy-translation-t5-small-bahasa-cased-v4')
        Check available models at `malaya.augmentation.abstractive.available_huggingface()`.
    lang: str, optional (default='ms')
        Input language, only accept `ms` or `en`.
    force_check: bool, optional (default=True)
        Force check model one of malaya model.
        Set to False if you have your own huggingface model.

    Returns
    -------
    result: malaya.torch_model.huggingface.Generator
    """
```

So you can choose either `ms` or `en` input language, for mixed language, it is save to choose both.

In [14]:
model = malaya.augmentation.abstractive.huggingface(model = 'mesolitica/finetune-noisy-translation-t5-small-bahasa-cased-v4')

### Generate

```python
def generate(self, strings: List[str], return_generate=False, **kwargs):
    """
    Generate texts from the input.

    Parameters
    ----------
    strings : List[str]
    **kwargs: vector arguments pass to huggingface `generate` method.
        Read more at https://huggingface.co/docs/transformers/main_classes/text_generation

    Returns
    -------
    result: List[str]
    """
```

In [9]:
string = 'saya suka makan ayam dan ikan'
string2 = 'saya sebenarnya tak suka sangat dekat lelaki tu, ketiak masam sebab tak mandi'
string3 = 'Perdana Menteri berkata, beliau perlu memperoleh maklumat terperinci berhubung isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut. Bagaimanapun, beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan boleh berfungsi dengan baik.'

In [10]:
model.generate([string, string2, string3], max_length = 256)

['Suka makan ayam ikan',
 'Aku sebenarnya tak suka sangat dekat lelaki tu, ketiak masam sebab tak mandi',
 'Perdana Menteri berkata, beliau perlu mendapatkan maklumat terperinci mengenai isu tersebut sebelum kerajaan dapat mengambil sebarang tindakan lanjut. Bagaimanapun, beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan boleh berfungsi dengan baik.']

### Good thing about HuggingFace

In `generate` method, you can do greedy, beam, sampling, nucleus decoder and so much more, read it at https://huggingface.co/blog/how-to-generate

In [11]:
outputs = model.generate([string, string2, string3], 
                         max_length = 100, do_sample=True, top_k=100, top_p=0.95, temperature=0.7,
                         num_return_sequences=3)
outputs

['sy suka makan ayam dan ikan',
 'aku suka makan ayam ikan',
 'Suka makan ayam dan ikan',
 'i actually tak berapa suka sangat dekat laki tu ketiak masam sebab tak mandi',
 'sebenarnya i tak suka sangat dkt lelaki tu tu ketiak masam sbb x mandi',
 'aku sebenarnya tak suka sangat dekat lelaki tu, ketiak masam sebab tak mandi',
 'Perdana Menteri berkata, beliau perlu mendapatkan maklumat terperinci berhubung isu tersebut sebelum kerajaan dapat mengambil sebarang tindakan lanjut. Apa-apa pun, beliau yakin masalah dapat diselesaikan dan pentadbiran kerajaan dapat berfungsi dengan baik.',
 'Perdana Menteri berkata, beliau perlu mendapatkan maklumat terperinci tentang isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut. Bagaimanapun beliau yakin masalah tersebut dapat diselesaikan dan pentadbiran kerajaan boleh berfungsi dengan baik.',
 'Perdana Menteri berkata, beliau perlu mendapatkan maklumat yang terperinci mengenai isu tersebut sebelum kerajaan dapat mengambil sebaran

You can choose different generating method and still maintain the same polarity.

In [15]:
strings = [
    'bodoh betul kerajaan ni, mana kebebasan bersuara',
    'Perbincangan khas itu juga bertujuan bagi Seri Paduka mendapat pandangan Raja-Raja Melayu untuk membolehkan baginda membuat keputusan yang terbaik demi kepentingan dan kesejahteraan negara serta rakyat',
    'semalam saya buka tv tengok berita, katanya ada tanah runtuh',
    'i hate him so much, perangai teruk!'
]

In [21]:
outputs = model.generate(strings, 
                         max_length = 100, do_sample=True, top_k=100, top_p=0.95, temperature=0.8,
                         num_return_sequences=3)
outputs

['bodoh betul kerajaan ni, mana kebebasan bersuara',
 'bodoh betul kerajaan ni mana free speech',
 'bodoh betul kerajaan ni, mana kebebasan bersuara',
 'Perbincangan khas juga bertujuan bagi Seri Paduka mendapat pandangan Raja2 Melayu untuk membolehkan baginda membuat keputusan yang terbaik demi kepentingan dan kesejahteraan negara serta rakyat',
 'Perbincangan khas juga bertujuan bagi Seri Paduka mendapat pandangan Raja2 Melayu bagi membolehkan baginda membuat keputusan yg terbaik demi kepentingan dan kesejahteraan negara serta rakyat',
 'Perbincangan khas juga bertujuan bagi Seri Paduka mendapat pandangan Raja-Raja Melayu agar beliau membuat keputusan yang terbaik demi kepentingan dan kesejahteraan Negara dan rakyat',
 'semalam buka tv tgk berita, dia kata ada tanah runtuh',
 'semalam buka tv tgk news katanya ada tanah runtuh',
 'semalam aku bukak tv tengok news, katanya ada landslide',
 'Aku benci dia sangat, perangai buruk!',
 'i hate him so much, teruknya perangai ye!',
 'u hate h